In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy.optimize import fmin_ncg

In [2]:
data = pd.read_csv('data/heart.csv')
sample_input = np.c_[data['age'], data['sex'], data['cp'], data['trestbps'], data['chol'], data['fbs']
                    , data['restecg'], data['thalach'], data['exang'], data['oldpeak'], data['slope']
                    , data['ca'], data['thal']]
sample_output = np.c_[data['target']]
data.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1


In [3]:
# data for our model
X = np.c_[np.ones((sample_input.shape[0],1)),sample_input]
Y = np.c_[sample_output]
initial_theta = np.zeros(X.shape[1])

In [4]:
# Activation function used to map any real value between 0 and 1
def sigmoid(z):
    return 1/(1+np.exp(-z) )

In [5]:
# Computes the cost function for all the training samples
def cost_function(theta,x,y):
    theta = theta.reshape(-1,1)
    m = y.size
    h = sigmoid((x.dot(theta)).reshape(-1,1))
    j = -(1/m) * (np.log(h).T.dot(y) + np.log(1-h).T.dot((1-y)))
    return j

In [6]:
# Computes the gradient of the cost function at the point theta
def gradient(theta,x,y):
    theta = theta.reshape(-1,1)
    m = y.size
    h = sigmoid((x.dot(theta)).reshape(-1,1))
    grad = (1/m) * X.T.dot(h-y)
    return grad.flatten()

In [7]:
cost = cost_function(initial_theta,X,Y)
grad = gradient(initial_theta,X,Y)
print('Cost:', cost)
print('Gradient:', grad)

Cost: [[0.69314718]]
Gradient: [-4.45544554e-02 -1.40429043e+00  3.46534653e-02 -2.65676568e-01
 -4.60066007e+00 -8.77557756e+00 -1.65016502e-03 -5.94059406e-02
 -1.14702970e+01  8.74587459e-02  2.02310231e-01 -1.68316832e-01
  1.66666667e-01  1.65016502e-03]


In [8]:
def fit(theta,x,y):
    optimized = fmin_ncg(f=cost_function, x0=theta, fprime=gradient, args=(x, y))
    return optimized

In [9]:
parameter = fit(initial_theta,X,Y)

Optimization terminated successfully.
         Current function value: 0.353916
         Iterations: 14
         Function evaluations: 22
         Gradient evaluations: 120
         Hessian evaluations: 0


In [10]:
def predict(theta,x):
    p = sigmoid(x.dot(theta.T))>=0.60
    return p.astype('int')

In [11]:
# Predict using the optimized Theta values from above (parameter).
print(parameter)
sigmoid(X[130].dot(parameter.T))

[ 0.04499281  0.01118623 -1.31457262  0.74329776 -0.01422624 -0.00268146
 -0.07832454  0.55377982  0.0306895  -0.76614587 -0.48548162  0.51012683
 -0.6814552  -0.94701845]


0.9651558420441141

In [12]:
predict(parameter,X[130])

1

In [13]:
p = predict(parameter,X)
print('Train accuracy : {}%'.format(100*sum(p == Y.ravel())/p.size))

Train accuracy : 86.46864686468646%
